In [5]:
import os
import sys
import pandas as pd
import openpyxl

import time
from timeit import default_timer as timer
from datetime import datetime
from datetime import datetime, date, timedelta
from bs4 import BeautifulSoup

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.options import Options


from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# NAVER BLOG Crawling
- title/url 

- ver02-2

In [15]:
# start_time = timer()

# 검색어 입력
# query = input("검색어 : ")
# location = str(query)

query = input()
# f = open("crawling/station.txt", 'r', encoding='utf-8')
# keywords = f.readlines()
# keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')

# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []
# start_crawling_time = timer()
# print(start_crawling_time)
# try:
# 검색 URL 생성
base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
driver.get(base_url)
time.sleep(1)  # 페이지가 로드될 때까지 대기

html = driver.page_source
soup = BeautifulSoup(html, "html.parser")
search_number = soup.find(class_="search_number")
total_num = search_number.text.replace("건",'').replace(",",'')
total_num = int(total_num)
end_page=total_num//7+1

# 페이지마다 게시물 가져오기
for page_num in range(1, total_num // 7 + 2):
    try :
        search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(search_url)
        time.sleep(0.5)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")

        # 블로그 포스트 데이터 가져오기
        posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
        posts_date_data = soup.select('div.info_post > div.writer_info')

        # 타이틀 수가 7개가 아니면 루프를 빠져나오기
        if len(posts_data) >= 6:
            for post, post_date in zip(posts_data, posts_date_data):
                title = post.text.replace('\n', '').strip()
                href = post.attrs['href']
                date = post_date.find('span', 'date').text.strip()
                try :
                    date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                except:
                    date_timestamp = endDate
                crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
        else :
            break
    except Exception as e:
        print(f"에러 발생: {e}")
# end_crawling_time = timer()
# print(end_crawling_time)
# print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.close()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))

In [16]:
crawling_blog_data

[{'location': '강남역',
  'title': '강남역 임플란트 2차수술까지 불편함 없이 진행하려면',
  'url': 'https://blog.naver.com/hyangwons/223454267378',
  'posted_date': '2024-05-22'},
 {'location': '강남역',
  'title': '강남역치과 발치 후 빠른 식립하는 임플란트',
  'url': 'https://blog.naver.com/zmfflr1026/223454991228',
  'posted_date': '2024-05-22'},
 {'location': '강남역',
  'title': '강남역치과 빠진 치아가 있을때',
  'url': 'https://blog.naver.com/randy1313/223453752301',
  'posted_date': '2024-05-21'},
 {'location': '강남역',
  'title': '강남역임플란트 치아상실의 대처 방안으로',
  'url': 'https://blog.naver.com/happ0217/223455873820',
  'posted_date': '2024-05-24'},
 {'location': '강남역',
  'title': '강남역임플란트 탈락한 치아를 대체하는 방법',
  'url': 'https://blog.naver.com/thfdl0216/223454439111',
  'posted_date': '2024-05-22'},
 {'location': '강남역',
  'title': '강남역 점심 맛집 내돈내산 트라가 후기',
  'url': 'https://blog.naver.com/luckyjieun_/223455125785',
  'posted_date': '2024-05-22'},
 {'location': '강남역',
  'title': '강남역 pt 건강하게 운동하는 법',
  'url': 'https://blog.naver.com/ohmytt_/223449936110',


In [17]:
print(len(crawling_blog_data))
print(search_number)

3997
<em class="search_number">10,609건</em>


In [26]:
# start_time = timer()
# 검색어 입력
# query = input("검색어 : ")
# location = str(query)

f = open("crawling/station.txt", 'r', encoding='utf-8')
keywords = f.readlines()
keywords_data = []

# 조회 기간 설정 : 7일(days에서 수정 가능)
# startDate=7일전 날짜, endDate=오늘 날짜
date = datetime.now()
startDate= (date+timedelta(days=-7)).strftime('%Y-%m-%d')
endDate = (datetime.now()).strftime('%Y-%m-%d')

# 브라우저 옵션 설정
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_argument('headless')
options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/125.0.0.0 Safari/537.36')
# 브라우저 초기화
driver = webdriver.Chrome(options=options)

crawling_blog_data = []
for query in keywords:
    # start_crawling_time = timer()
    # print(start_crawling_time)
    try:
        # 검색 URL 생성
        base_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
        driver.get(base_url)
        time.sleep(1)  # 페이지가 로드될 때까지 대기

        html = driver.page_source
        soup = BeautifulSoup(html, "html.parser")
        search_number = soup.find(class_="search_number")
        total_num = search_number.text.replace("건",'').replace(",",'')
        total_num = int(total_num)
        end_page=total_num//7+1

        # 페이지마다 게시물 가져오기
        for page_num in range(1, total_num // 7 + 2):
            search_url = f"https://section.blog.naver.com/Search/Post.naver?pageNo={page_num}&rangeType=WEEK&orderBy=sim&startDate={startDate}&endDate={endDate}&keyword={query}"
            driver.get(search_url)
            time.sleep(1)  # 페이지가 로드될 때까지 대기

            html = driver.page_source
            soup = BeautifulSoup(html, "html.parser")

            # 블로그 포스트 데이터 가져오기
            posts_data = soup.select('div.info_post > div.desc > a.desc_inner')
            posts_date_data = soup.select('div.info_post > div.writer_info')

            # 타이틀 수가 7개가 아니면 루프를 빠져나오기
            if len(posts_data) >= 6:
                for post, post_date in zip(posts_data, posts_date_data):
                    title = post.text.replace('\n', '').strip()
                    href = post.attrs['href']
                    date = post_date.find('span', 'date').text.strip()
                    try :
                        date_timestamp = datetime.strptime(date, '%Y. %m. %d.').strftime('%Y-%m-%d')
                    except:
                        date_timestamp = endDate
                    crawling_blog_data.append({"location": query, "title": title, "url": href, "posted_date": date_timestamp})
            else :
                break
    except Exception as e:
        print(f"에러 발생: {e}")

    # end_crawling_time = timer()
    # print(end_crawling_time)
    # print("working_crawling_time: {} sec".format(end_crawling_time-start_crawling_time))


# 브라우저 종료
driver.quit()
# end_time = timer()
# print("working_time: {} sec".format(end_time-start_time))

에러 발생: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome-headless-shell=125.0.6422.61)
Stacktrace:
	GetHandleVerifier [0x00007FF625031F22+60322]
	(No symbol) [0x00007FF624FACE99]
	(No symbol) [0x00007FF624E67EBA]
	(No symbol) [0x00007FF624E4F1CC]
	(No symbol) [0x00007FF624E4F090]
	(No symbol) [0x00007FF624E6A4E1]
	(No symbol) [0x00007FF624EFB359]
	(No symbol) [0x00007FF624EDBFC3]
	(No symbol) [0x00007FF624EA9617]
	(No symbol) [0x00007FF624EAA211]
	GetHandleVerifier [0x00007FF62534946D+3301613]
	GetHandleVerifier [0x00007FF625393693+3605267]
	GetHandleVerifier [0x00007FF625389410+3563664]
	GetHandleVerifier [0x00007FF6250E42F6+790390]
	(No symbol) [0x00007FF624FB74DF]
	(No symbol) [0x00007FF624FB33D4]
	(No symbol) [0x00007FF624FB3562]
	(No symbol) [0x00007FF624FA2F6F]
	BaseThreadInitThunk [0x00007FFB6884257D+29]
	RtlUserThreadStart [0x00007FFB69CCAA48+40]

에러 발생: Message: disconnecte

In [27]:
df = pd.DataFrame(crawling_blog_data)
df.sort_values("posted_date", axis = 0, ascending=False, inplace=True)


In [29]:
file_name = f"naver_blog_list_{startDate}-{endDate}.csv"
df.to_csv(file_name, index=False)
data = pd.read_csv(file_name)
data

,location,title,url,posted_date
0,회기역,회기역 월주차 2번출구 인근 편리한곳,https://blog.naver.com/plqh9295/223455804000,2024-05-23
1,회기역,"회기동 월주차 _ 회기역 도보 5분거리의 지정석,단일 주차장",https://blog.naver.com/zoomansa1/223455635624,2024-05-23
2,회기역,"행복주택 예비입주자 청약 신청 총정리 (월세 6만원부터,초역세권, 가좌역/회기역)",https://blog.naver.com/newchorok/223455224162,2024-05-23
3,회기역,회기역 맛집 회기왕갈비탕 점심 혼밥하기 좋은 외대 맛집,https://blog.naver.com/kwakyoukyeong/223455354721,2024-05-23
4,회기역,회기역 근처 맛집 밥집 '큐큐면관' - 5년 거주민이 데려가준 곳,https://blog.naver.com/joppdbsk/223450702877,2024-05-23
...,...,...,...,...
416,회기역,동대문구 실용음악학원 동대문구 드럼학원 동대문구 보컬학원 동대문구 댄스학원...,https://blog.naver.com/fwinds/223448265584,2024-05-16
417,회기역,24년 4월 일기(부동산 임장/초보 체험단/ADSP 독학/프리미엄 막걸리),https://blog.naver.com/nongnogne/223444593219,2024-05-16
418,회기역,경희대카페 스터디하기좋은 대형카페 _ 8번가 8street,https://blog.naver.com/mydreameruza/223448746614,2024-05-16
419,회기역,동대문미용실 악성곱슬 볼륨매직 예스헤어 만족합니다,https://blog.naver.com/co1823/223447979677,2024-05-16
